# Wordle Solver Application

This notebook implements a comprehensive Wordle Solver application with an interactive command-line interface. The solver uses advanced algorithms to suggest optimal guesses based on letter frequency analysis and word commonality, helping users solve Wordle puzzles efficiently.

## Features
- Interactive CLI with colorful Rich-based interface
- Advanced guess suggestion algorithm
- Letter frequency analysis
- Game history and performance tracking
- Support for multiple rounds
- Comprehensive error handling

In [ ]:
!pip install rich

In [ ]:
import os
import random
from collections import Counter, defaultdict
from typing import List, Set, Dict, Tuple, Optional
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
from rich.prompt import Prompt, Confirm
from rich import 

In [ ]:
class WordList:
    def __init__(self, all_words_file: str, common_words_file: str):
        self.all_words = self._load_words(all_words_file)
        self.common_words = set(self._load_words(common_words_file))
        self.current_candidates = self.all_words.copy()
        
    def _load_words(self, filename: str) -> Set[str]:
        try:
            with open(filename, 'r') as f:
                return {word.strip().upper() for word in f.readlines() if len(word.strip()) == 5}
        except FileNotFoundError:
            rprint(f"[red]Warning: Could not find {filename}. Using fallback word list.[/red]")
            return self._get_fallback_words()
    
    def _get_fallback_words(self) -> Set[str]:
        return {
            'ADIEU', 'AUDIO', 'HOUSE', 'PLANT', 'WORLD', 'LIGHT', 'SOUND', 'WATER',
            'MUSIC', 'SMILE', 'BRAIN', 'HEART', 'DRIVE', 'CLOUD', 'SMART', 'QUIET',
            'STORM', 'PEACE', 'ROUND', 'STONE', 'QUICK', 'BREAD', 'DREAM', 'SPACE',
            'GRACE', 'PLACE', 'CLEAN', 'FRESH', 'GREAT', 'SMALL', 'NEVER', 'HAPPY',
            'YOUNG', 'STORY', 'POWER', 'MONEY', 'RIGHT', 'NIGHT', 'SWEET', 'CLEAR'
        }
    
    def filter_words(self, guess: str, result: str) -> None:
        new_candidates = set()
        
        for word in self.current_candidates:
            if self._word_matches_result(word, guess, result):
                new_candidates.add(word)
        
        self.current_candidates = new_candidates
    
    def _word_matches_result(self, word: str, guess: str, result: str) -> bool:
        word_letters = list(word)
        guess_letters = list(guess)
        
        # Check green letters (correct position)
        for i, (g_letter, r) in enumerate(zip(guess_letters, result)):
            if r == 'G' and word_letters[i] != g_letter:
                return False
            elif r == 'G':
                word_letters[i] = None
                guess_letters[i] = None
        
        # Check yellow and black letters
        for i, (g_letter, r) in enumerate(zip(guess, result)):
            if guess_letters[i] is None:  # Already processed as green
                continue
                
            if r == 'Y':
                if g_letter not in word_letters or word_letters[i] == g_letter:
                    return False
                word_letters[word_letters.index(g_letter)] = None
            elif r == 'B':
                if g_letter in word_letters:
                    return False
        
        return True
    
    def get_candidates(self) -> Set[str]:
        return self.current_candidates
    
    def reset_candidates(self) -> None:
        self.current_candidates = self.all_words.copy()
    
    def is_valid_word(self, word: str) -> bool:
        return word.upper() in self.all_words

In [ ]:
class WordleSolver:
    def __init__(self, word_list: WordList):
        self.word_list = word_list
        self.guesses = []
        self.results = []
        self.game_won = False
        self.max_guesses = 6
        
    def add_guess(self, guess: str, result: str) -> None:
        self.guesses.append(guess.upper())
        self.results.append(result.upper())
        self.word_list.filter_words(guess.upper(), result.upper())
        
        if result.upper() == 'GGGGG':
            self.game_won = True
    
    def suggest_next_guess(self) -> str:
        candidates = self.word_list.get_candidates()
        
        if not candidates:
            return "No valid words found"
        
        if len(candidates) == 1:
            return list(candidates)[0]
        
        # If first guess, use optimal starting words
        if len(self.guesses) == 0:
            optimal_starters = ['ADIEU', 'AUDIO', 'AROSE', 'RAISE', 'SOARE']
            for starter in optimal_starters:
                if starter in candidates:
                    return starter
        
        # Score words based on letter frequency and commonality
        scored_words = []
        letter_freq = self._calculate_letter_frequency(candidates)
        
        for word in candidates:
            score = self._score_word(word, letter_freq, candidates)
            scored_words.append((word, score))
        
        # Sort by score (descending) and prefer common words
        scored_words.sort(key=lambda x: (x[1], x[0] in self.word_list.common_words), reverse=True)
        
        return scored_words[0][0]
    
    def _calculate_letter_frequency(self, candidates: Set[str]) -> Dict[str, int]:
        letter_count = Counter()
        for word in candidates:
            for letter in set(word):  # Count each letter once per word
                letter_count[letter] += 1
        return letter_count
    
    def _score_word(self, word: str, letter_freq: Dict[str, int], candidates: Set[str]) -> float:
        # Base score from letter frequency
        score = sum(letter_freq.get(letter, 0) for letter in set(word))
        
        # Bonus for common words
        if word in self.word_list.common_words:
            score *= 1.5
        
        # Penalty for repeated letters (less information gain)
        if len(set(word)) < 5:
            score *= 0.8
        
        return score
    
    def get_remaining_guesses(self) -> int:
        return self.max_guesses - len(self.guesses)
    
    def is_game_over(self) -> bool:
        return self.game_won or len(self.guesses) >= self.max_guesses
    
    def reset_game(self) -> None:
        self.guesses = []
        self.results = []
        self.game_won = False
        self.word_list.reset_candidates()
    
    def get_game_stats(self) -> Dict:
        return {
            'guesses_used': len(self.guesses),
            'game_won': self.game_won,
            'remaining_candidates': len(self.word_list.get_candidates()),
            'guesses': self.guesses.copy(),
            'results': self.results.copy()
        

In [ ]:
class UserInterface:
    def __init__(self):
        self.console = Console()
        
    def display_welcome(self) -> None:
        welcome_text = """
🎯 Welcome to the Wordle Solver! 🎯

This application will help you solve Wordle puzzles by suggesting optimal guesses
based on advanced algorithms and letter frequency analysis.

📋 How to use:
• Enter your guess when prompted
• Enter the result using: G (Green), Y (Yellow), B (Black)
• Follow the suggestions for optimal gameplay

🎮 Let's start solving!
        """
        
        panel = Panel(welcome_text, title="Wordle Solver", border_style="green")
        self.console.print(panel)
    
    def display_game_state(self, solver: WordleSolver) -> None:
        if not solver.guesses:
            return
            
        table = Table(title="Game Progress", show_header=True, header_style="bold blue")
        table.add_column("Guess #", style="cyan", width=8)
        table.add_column("Word", style="white", width=10)
        table.add_column("Result", style="white", width=15)
        
        for i, (guess, result) in enumerate(zip(solver.guesses, solver.results), 1):
            colored_result